In [1]:
%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=9000)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy

#import imp
from scan_gym import envs
#imp.reload(envs)


seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

GPUs:  1


In [2]:
current_path = os.getcwd()
#sys.path.append(os.path.join(current_path, 'scan_gym/scan_gym/envs/ScannerEnv/') )
#import scanner_env
params_file = os.path.join(current_path, 'params.json') 
pm=json.load(open(params_file))

In [3]:
#env = gym.make('GatekeeperEnv-v0')
#precomputed inliers ratios for all combinations of images
'''envs_paths = [os.path.join(current_path, 'plants_dataset/scan_plant001/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant002/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant003/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant004/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant005/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant006/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant007/v01/inliers_ratios.npy'),
             ]

data_paths = [os.path.join(current_path, 'plants_image_dataset/scan_plant001/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant002/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant003/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant004/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant005/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant006/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant007/v01/gray_imgs_224/')
            ]'''



envs_paths = [os.path.join(current_path, 'plants_dataset/scan_plant001/v01/inliers_ratios.npy'),
                os.path.join(current_path, 'plants_dataset/scan_plant002/v01/inliers_ratios.npy')
             ]

data_paths = [os.path.join(current_path, 'plants_image_dataset/scan_plant001/v01/gray_imgs_224/'),
              os.path.join(current_path, 'plants_image_dataset/scan_plant002/v01/gray_imgs_224/')
            ]




#env = suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.03,
#                                                 'inliers_ratios_file':os.path.join(current_path, 'plants_image_dataset/scan_plant001/v01/inliers_ratios.npy'),
#                                                  'dataset_path':os.path.join(current_path, 'plants_image_dataset/scan_plant001/v01/gray_imgs/')})
#tf_env = tf_py_environment.TFPyEnvironment(env)


#tf_agents.system.multiprocessing.enable_interactive_mode()#parallel environment in notebooks
#envs_callables = [lambda: suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.2,'inliers_ratios_file':x}) for x in envs_paths ]
#tf_env = tf_py_environment.TFPyEnvironment( parallel_py_environment.ParallelPyEnvironment(envs_callables) )



envs = [suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.03,
        'inliers_ratios_file':il,'dataset_path':dp}) 
        for il,dp in zip(envs_paths,data_paths)]

tf_env = tf_py_environment.TFPyEnvironment( batched_py_environment.BatchedPyEnvironment(envs,multithreading=True))#, isolation=True  )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
tf_env.observation_spec()

(BoundedTensorSpec(shape=(224, 224, 2), dtype=tf.float32, name='observation/tuple_0', minimum=array(0., dtype=float32), maximum=array(255., dtype=float32)),
 BoundedTensorSpec(shape=(5,), dtype=tf.float32, name='observation/tuple_1', minimum=array([-200.,    0.,    0.,  -20.,    0.], dtype=float32), maximum=array([200.,   8.,  10.,  20.,   1.], dtype=float32)))

In [5]:
def image_layers():
    input_img = keras.layers.Input(shape=(224,224,2))
    preprocessing = keras.layers.Lambda(lambda x: x / 255.)(input_img)
    x = keras.layers.Conv2D(filters=32,
                            kernel_size=8,
                            padding="same",
                            strides=(4,4),
                            activation='relu')(preprocessing)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=4,
                            padding="same",
                            strides=(2,2),
                            activation='relu')(x)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=3,
                            padding="same",
                            strides=(1,1),
                            activation='relu')(x)
    x = keras.layers.Flatten()(x)
    model = keras.models.Model(inputs=input_img,outputs=x)
    return model
    
    
    
#scale range 0 to 1
oldmin = tf_env.observation_spec()[0].minimum
oldmax = tf_env.observation_spec()[0].maximum
    
def input_vect_layers():
    input_ = keras.layers.Input(shape=(5,))
    preprocessing = keras.layers.Lambda(lambda x: ((x-oldmin)*(1.- 0.)/(oldmax-oldmin)) + 0. )(input_)
    #x = keras.layers.Dense(4)(preprocessing)
    return keras.models.Model(inputs=input_,outputs=preprocessing)

In [6]:
#network
preprocessing_layers=(image_layers(),input_vect_layers())
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
fc_layer_params = (512,)#pm['model']['fc_layer_params']

q_net = QNetwork(
tf_env.observation_spec(),
tf_env.action_spec(),
preprocessing_layers=preprocessing_layers,
preprocessing_combiner=preprocessing_combiner,
fc_layer_params=fc_layer_params)

In [7]:
#agent
train_step = tf.Variable(0)
#optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
#            epsilon=0.00001, centered=True)
'''lr_decay =  keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=0.005, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.0005)
optimizer = keras.optimizers.Adam(learning_rate=lambda: lr_decay(train_step))'''


optimizer = keras.optimizers.Adam(learning_rate=pm['model']['learning_rate'])

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.01) # final ε

agent = DqnAgent(tf_env.time_step_spec(),
                tf_env.action_spec(),
                q_network=q_net,
                optimizer=optimizer,
                target_update_period=pm['agent']['target_update_period'],
                td_errors_loss_fn= element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                gamma=pm['agent']['gamma'], # discount factor
                train_step_counter=train_step,
                epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()


In [8]:
#Replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec= agent.collect_data_spec,
    batch_size= tf_env.batch_size,
    max_length=pm['rbuffer']['max_length'])

In [9]:
#observer
#observer is just a function (or a callable object) that takes a trajectory argument,
#add_method() method (bound to the replay_buffer object) can be used as observer
replay_buffer_observer = replay_buffer.add_batch

In [10]:
#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(batch_size=len(envs)),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(batch_size=len(envs)),
tf_metrics.MaxReturnMetric(batch_size=len(envs)),
tf_metrics.MinReturnMetric(batch_size=len(envs)),   
]


'''#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(),
tf_metrics.MaxReturnMetric(),
tf_metrics.MinReturnMetric(),   
]'''

'#observer for training metrics\ntraining_metrics = [\ntf_metrics.NumberOfEpisodes(),\ntf_metrics.AverageEpisodeLengthMetric(),\ntf_metrics.EnvironmentSteps(),\ntf_metrics.AverageReturnMetric(),\ntf_metrics.MaxReturnMetric(),\ntf_metrics.MinReturnMetric(),   \n]'

In [11]:
#custom observer
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [12]:
#Collect Driver
update_period = pm['collect_driver']['num_steps'] # train the model every x steps
collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect x steps for each training iteration

#+ training_metrics,

In [13]:
# random policy driver to start filling the buffer
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)
#, ShowProgress(ns)],
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [14]:
#use buffer as tf API dataset ()
dataset = replay_buffer.as_dataset(
        sample_batch_size=pm['rbuffer']['sample_batch_size'],
        num_steps=2,
        num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [15]:
log_folder = "logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_dir = os.path.join(current_path, log_folder)    
train_summary_writer = tf.summary.create_file_writer(
            train_dir, flush_millis=10000)
#train_summary_writer.set_as_default()

In [16]:
#convert main functions to tensorflow functions to speed up training
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [17]:
#@tf.function
def train_agent(n_iterations):
    #reset metrics
    for m in training_metrics:
        m.reset()
    time_step = None
    policy_state = ()#agent.collect_policy.get_initial_state(tf_env.batch_size)
    agent.train_step_counter.assign(0)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                tf.summary.histogram("train/hist",q_net.layers[0].weights[0],step=tf.cast(agent.train_step_counter,tf.int64))
                tf.summary.scalar('train_loss', train_loss.loss.numpy(), step=tf.cast(agent.train_step_counter,tf.int64))
                #train_summary_writer.flush()
                for train_metric in training_metrics:
                    train_metric.tf_summaries(train_step=tf.cast(agent.train_step_counter,tf.int64), step_metrics=training_metrics[:])
                train_summary_writer.flush()

In [18]:
# Launch TensorBoard with objects in the log directory
# This should launch tensorboard in your browser, but you may not see your metadata.
#%tensorboard --logdir=logs --reload_interval=15

In [19]:
#tf.summary.scalar('avgreturn', training_metrics[3].result().numpy(), step=tf.cast(agent.train_step_counter,tf.int64))

In [ ]:
%time train_agent(pm['misc']['n_iterations'])

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


70019 loss:10.9513179

In [ ]:
replay_buffer.num_frames()

In [ ]:
env = suite_gym.load('ScannerEnv-v0',gym_kwargs={'setpoint':0.03,
                                                 'inliers_ratios_file':os.path.join(current_path, 'plants_image_dataset/scan_plant002/v01/inliers_ratios.npy'),
                                                  'dataset_path':os.path.join(current_path, 'plants_image_dataset/scan_plant002/v01/gray_imgs_224/')})
test_env = tf_py_environment.TFPyEnvironment( env )

state = test_env.reset()
#print(state.observation[0].numpy())
last_position = env.current_position
cont = 0
time_steps = 3000
#obs = deque(maxlen=time_steps)

for i in range(time_steps):
    #obs.append(state.observation[0].numpy())
    cont +=1
    action_step = agent.policy.action(state)
    next_state = test_env.step(action_step.action)
    print(cont,':',last_position,env.previous_match_idx,state.observation[1].numpy(),env.action2move(action_step.action.numpy(),state.observation[1].numpy()[0][-1]) ) #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    state = copy.deepcopy(next_state)
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_images)
print(sorted(env.kept_images))
print(env.total_reward)
print(len(env.kept_images))
print(env.total_moves)

In [ ]:
#batched_py_environment.BatchedPyEnvironment(envs=[suite.load(...) for _ in range(n)])

In [ ]:
#q_net.layers[0].weights[0].shape